In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
import numpy as np

#  Briefly describe all the issues you can find in data gathering and what they are caused by.

## Code from the DTVC_Week 2_Exercise notebook

In [ ]:
def scrape_aldi_jobs(starting_page = 'head-office'):
    url = "https://www.aldirecruitment.co.uk/" + starting_page
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    links = []
    for link in soup.findAll('a'):
        new_link = link.get('href')
        if new_link != None and new_link.startswith('/head-office/'):
            new_link = new_link.replace('/head-office','')
            if new_link != '/':
                links.append(new_link)
    
    department = []
    titles = []
    ubs = []
    lbs = []
    hours = []
    for link in links:
        category_url = url + link
        page = requests.get(category_url)
        soup = BeautifulSoup(page.content, "html.parser")
        postings = soup.findAll("div", class_="c-career--dropdown")
        for posting in postings:
            ## Also grabbing the department information
            dep_name = link.replace('-',' ').replace('/','')
            department.append(dep_name)
            titles.append(posting.find("div", class_="c-career--dropdown__content").find('h2').text)
            details = posting.findAll("div", class_="c-job-details__content")
            for detail in details:
                detail_title = detail.find('span', class_="c-job-details__title").text
                detail_text = detail.find('div', class_="c-job-details__text").text
                if detail_title == 'Salary':
                    temp = detail_text.replace(',','')
                    temp = temp.replace('-','')
                    temp = temp.split()
                    salary_numbers = [float(s[1:]) for s in temp if s.startswith('£')]
                    ## Salary may not be specified
                    if len(salary_numbers) > 0:
                        ## Salaries are sometimes specified as per week instead of per year
                        if 'per' in temp and 'week' in temp:
                            salary_numbers = [salary*52 for salary in salary_numbers]
                        ubs.append(max(salary_numbers))
                        lbs.append(min(salary_numbers))
                    else:
                        ubs.append(None)
                        lbs.append(None)
                ## Some postings say "Benefits" instead of "Hours and benefits", and sometimes the spelling is capitalized differently
                elif detail_title.lower() == 'hours and benefits' or detail_title.lower() == 'benefits':
                    ## Some postings do not specify a number of hours per week
                    work_time = None
                    for s in detail_text.split():
                        if '-hour' in s:
                            work_time = s
                            ## Some postings write, e.g., 40-hour per week, some 40-hours per week
                            if '-hours' in s:
                                work_time = int(work_time.replace('-hours',''))
                            else:
                                work_time = int(work_time.replace('-hour',''))
                    hours.append(work_time)
                        
    job_data = pd.DataFrame(
        {'Department': department,
         'Job title': titles,
         'Salary lower': lbs,
         'Salary upper': ubs,
         'Weekly hours': hours
        })
    return job_data

In [ ]:
jobs_data = scrape_aldi_jobs()
jobs_data

## Errors appeared in the data gathering function

- The values of lower and upper salaries are not appropriately gathered (ex. 4.00 and 46435.00)
    - Reason: We didn't include a condition for salaries displayed in hour rate (ex. Customer Service Advisor, Quality Control Assistant)
    - Reason: We didn't include a condition for premium as an additional salary's element in the min function for the lower salary (ex. Quality Control Deputy)
    - Reason: Data Scientist posting has a typo in the lower salary. It has an extra space between comma and hundreds.
- The values of lower and upper salaries have NaN Values
    - Reason: The posting didn't specify a salary
- We have some NaN values in the weekly hours
    - Reason: The posting didn't specify weekly hours
    - Reason: The posting specify weekly hours without hyphen (ex. 40-hours per week, 40 hours per week)

In [ ]:
print(np.min(jobs_data['Weekly hours']), np.max(jobs_data['Weekly hours']), jobs_data['Weekly hours'].isna().sum())

In [ ]:
print(np.min(jobs_data['Salary lower']), np.max(jobs_data['Salary lower']), jobs_data['Salary lower'].isna().sum())

In [ ]:
print(np.min(jobs_data['Salary upper']), np.max(jobs_data['Salary upper']), jobs_data['Salary upper'].isna().sum())

# Rewrite the scraping function to overcome the issues you encountered above, collect additional information that could be sensible to analysis.

In [ ]:
def new_scrape_aldi_jobs(starting_page = 'head-office'):
    url = "https://www.aldirecruitment.co.uk/" + starting_page
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    links = []
    for link in soup.findAll('a'):
        new_link = link.get('href')
        if new_link != None and new_link.startswith('/head-office/'):
            new_link = new_link.replace('/head-office','')
            if new_link != '/':
                links.append(new_link)
    
    department = []
    titles = []
    ubs = []
    lbs = []
    hours = []
    
    years = []
    holiday = []
    medical = []   
    mothers = []
    fathers = []
    
    for link in links:
        category_url = url + link
        page = requests.get(category_url)
        soup = BeautifulSoup(page.content, "html.parser")
        postings = soup.findAll("div", class_="c-career--dropdown")
        for posting in postings:
            ## Also grabbing the department information
            dep_name = link.replace('-',' ').replace('/','')
            department.append(dep_name)
            titles.append(posting.find("div", class_="c-career--dropdown__content").find('h2').text)
            details = posting.findAll("div", class_="c-job-details__content")
            for detail in details:
                detail_title = detail.find('span', class_="c-job-details__title").text
                detail_text = detail.find('div', class_="c-job-details__text").text
    
                if detail_title == 'Salary':
                    temp = detail_text.replace(',','')
                    temp = temp.replace('-','')
                    temp = temp.split()
                    
                    ## Some postings state in each year the employee will reach his/her max salary            
                    yrs = None
                    if 'year' in temp:    
                        Index = temp.index('year')
                        temp_year = temp[Index+1]
                        if 'after' in temp:
                            Index = temp.index('after')
                            temp_year = temp[Index+1]
                        yrs =temp_year.replace('.','')
                        if ').' in temp_year:
                            yrs =temp_year.replace(').','')
                    if yrs == None:
                        years.append(yrs)
                    else:
                        years.append(int(yrs))
                                               
                    salary_numbers = [float(s[1:]) for s in temp if s.startswith('£')]
                    ## Salary may not be specified
                    if len(salary_numbers) > 0:
                        ## Salaries are sometimes specified as per week instead of per year
                        if 'per' in temp and 'week' in temp:
                            salary_numbers = [salary*52 for salary in salary_numbers]
                        ## Salaries are sometimes specified as per hours instead of per year
                        elif 'per' in temp and 'hour' in temp: 
                            salary_numbers = [(salary*40)*52 for salary in salary_numbers]    
                        ## Salaries are sometimes specified a premium which we want to exclude from lower salary 
                        if 'premium' in temp:
                            salary_numbers.sort()
                            salary_numbers = salary_numbers[1:]
                        ubs.append(max(salary_numbers))
                        lbs.append(min(salary_numbers))
                    else:
                        ubs.append(None)
                        lbs.append(None)
                ## Some postings say "Benefits" instead of "Hours and benefits", and sometimes the spelling is capitalized differently
                elif detail_title.lower() == 'hours and benefits' or detail_title.lower() == 'benefits':

                   ## Some postings provide medical insurance after some years or months 
                    med = 0
                    if 'medical' in detail_text.split():
                        med = 1         
                    medical.append(med)         
                    
                    ## Some postings do not specify a number of hours per week
                    work_time = None
                    for s in detail_text.split():
                        if '-hour' in s:
                            work_time = s
                            ## Some postings write, e.g., 40-hour per week, some 40-hours per week
                            if '-hours' in s:
                                work_time = int(work_time.replace('-hours',''))
                            else:
                                work_time = int(work_time.replace('-hour',''))
                    ## Some postings write, e.g., 40hour per week, some 40-hour per week 
                    if 'hours' in detail_text.split():        
                            Index = detail_text.split().index('hours')
                            work_time = int(detail_text.split()[Index-1])
                    hours.append(work_time)
                
                    ## Some postings provide maternity leave
                    mother = 0
                    if ('maternity' in detail_text.split()) or ('maternity,' in detail_text.split()):
                        mother = 1
                    mothers.append(mother)    
                    
                    ## Some postings provide paternity pay
                    father = 0
                    if ('paternity' in detail_text.split()) or ('paternity,' in detail_text.split()):
                        father = 1                 
                    fathers.append(father)                
                
                ## Some postings say "Holiday Allowance" instead of "Holiday", and sometimes the spelling is capitalized differently
                ## One posting have a typo of "Holiday allownance"
                elif detail_title.lower() == 'holiday' or detail_title.lower() == 'holiday allowance' or detail_title.lower() == 'holiday allownance':                                                            
                    off_time = None
                    ## Some postings specify a number of weeks for holiday days
                    if 'weeks’' in detail_text.split():        
                        Index = detail_text.split().index('weeks’')
                        off_time = int(detail_text.split()[Index-1])
                    ## Some postings specify a number of days for holiday days
                    elif 'days' in detail_text.split():
                        Index = detail_text.split().index('days')
                        off_time = (int(detail_text.split()[Index-1]))/7
                    elif 'days’' in detail_text.split():
                        Index = detail_text.split().index('days’')
                        off_time = (int(detail_text.split()[Index-1]))/7
                    holiday.append(off_time)
                    
    job_data = pd.DataFrame(
        {'Department': department,
         'Job title': titles,
         'Salary lower': lbs,
         'Salary upper': ubs,
         'Years to upper salary': years,
         'Weekly hours': hours,
         'Holiday weeks': holiday,
         'Maternity leave': mothers,
         'Paternity pay': fathers,
         'Medical insurance': medical         
        })
    return job_data

The additional information that we collected are the following:
- Holiday Allowance: We collected this data because we expect this to effect the salary given in each position. Also, this information is good to know in order to understand what our competitors give as common perk to potential candidates for each position. The first issue we encounter during the collection of this data was mostly due to differences in the title of the section in job description ('Holiday Allowance'/'Holiday'/'holiday allownance'(typo)). The second issue was that some postings specify the Holiday Allowance is terms of weeks and others in terms on days so we needed to convert that properly.
- Years to upper salary: We collected this data because we expect an effect of the amount of Max Salary someone can reach based on the years needed to reach it. The first issue we encounter here is that some postings have different sentence structure (ex. Quality Control Assistant & Quality Control Deputy) so we need to include this differences in the if statements. The second issue was that the numbers came with the following punctuation with them so we need to remove them in order to convert them to integers.
- Medical insurance: We collected this data in order to identify if it's common for our competitors to provide medical insurance as an extra benefit. We didn't encounter any issues in the collection of these data.
- Maternity leave & Paternity Pay: We collected this data in order to identify if it is common to provide extra perks regarding parenting in our competitors job descriptions. We didn't encounter any issues in the collection of these data.

The Medical insurance along with the Maternity leave & Paternity Pay was collected not to test how they affect the salaries but in order to have a more complete picture of the minimum requirement appear in our competitors as extra benefits.  
 

# Run your revised scraping function to collect information on postings and save the information as a csv file

In [ ]:
new_jobs = new_scrape_aldi_jobs()
new_jobs

In [ ]:
new_jobs.to_csv('Aldi_postings_revised.csv', index=False)

# Perform data pre-processing steps (cleansing, scaling, normalization, imputation)

## Data Cleansing

1. Checking for invalid values. Because the columns Salary lower and Salary upper contain only numeric values we will check for invalid ones by display them in a histogram and see if the plots are reasonable.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fig = plt.figure(figsize=(10,8))
ax0 = fig.add_subplot(1,2,1)
ax0.hist('Salary lower',  data= new_jobs, edgecolor='white')
ax0.set_title('Salary lower')
ax1 = fig.add_subplot(1,2,2)
ax1.hist('Salary upper', data= new_jobs, edgecolor='white')
ax1.set_title('Salary upper')
plt.show()

We detected that the majority of the minimum salary values are concentrated between 20000 and 60000. However, We noted that some lower salaries are at the range of 80000, so we need to examine if they are so high by mistake or have equally large upper salaries.

In [ ]:
new_jobs.loc[new_jobs['Salary lower']>=75000,]

Result: So the lower salaries are correctly gathered and the reason that they are separated from the others is that they are paired with positions of Senior Managers (higher paid positions).

1.Checking for invalid values also at columns Maternity leave, Paternity pay and Medical insurance which contain categorical values. We will check for invalid values by checking that their unique values are either 0 or 1. In Addition columns like Years to upper salary, Holiday weeks and weekly hours can have a limited range of values so we will check them as categorical variables.

In [ ]:
print('Maternity leave: ' ,new_jobs['Maternity leave'].unique())
print('Paternity pay: ', new_jobs['Paternity pay'].unique())
print('Medical insurance: ', new_jobs['Medical insurance'].unique())
print('Years to upper salary: ',new_jobs['Years to upper salary'].unique())
print('Holiday weeks: ', new_jobs['Holiday weeks'].unique())
print('Weekly hours: ', new_jobs['Weekly hours'].unique())

In general we didn't spot any extreme value that can be identified as invalid. However, in 'Years to upper salary' column the value 1 sims kind of small in regard to the others. Moreover, in 'Weekly hours' column, the value of 20 sees kind of small and looks suspicious if the corresponding job is not part-time. We will examine the two values further.

In [ ]:
new_jobs.loc[new_jobs['Years to upper salary']== 1,]

In [ ]:
new_jobs.loc[new_jobs['Weekly hours']== 20,]

2. Examine if there are duplicates in our dataset.

In [ ]:
double = new_jobs.duplicated()
print(double.any())

Results: There are no duplicates in the dataset because the double.any() function return False, which means there is none true value in the double variable. So we don't need to modify our dataset.

3. Checking for row records that have empty the salary columns.

In [ ]:
new_jobs.loc[new_jobs['Salary upper'].isna(),: ]

In [ ]:
new_jobs.loc[new_jobs['Salary lower'].isna(),: ]

We detect in the dataset that Dicterors' postings didn't specify the salary probably because the offers become personalized based on the candidate. So because they didn't specify the salary we will drop them from the data.

In [ ]:
for Index,title in enumerate(new_jobs['Job title']):
    if 'Director' in title:
        new_jobs.drop(index=Index, inplace=True)

In [ ]:
new_jobs.loc[new_jobs['Salary lower'].isna(),: ], new_jobs.loc[new_jobs['Salary upper'].isna(),: ] , len(new_jobs)

Results: We delete the rows which have postings for Director positions and we reduced our dataset from 99 rows to 95.

4. Examine if any one of the columns have the same value for every row.  

In [ ]:
new_jobs.nunique()

Results: None of the columns contained the same value in each row (all the values are higher than 1), so we don't need to modify the dataset.

## Data imputation for numerical variables

In the data cleansing process we detect that the weekly hours and the Years to upper salary columns have some missing values due to the fact that there are no specified in the job description. In order to fill those missing values we will randomly select one value from the jobs in the same departments which are closer to the missing job's description.  
The process is the following:
- Find the rows with the missing weekly hours
- Generate values based on different departments through random choice
- Assign the appropriate values to the missing rows (We did this manually because there are not many missing values. If the number of get bigger we will use the impute library from sklearn)
- Check if there are other missing values in this column using unique()

In [ ]:
new_jobs.loc[pd.isna(new_jobs['Weekly hours']),]

In [ ]:
np.random.seed(222)
random = new_jobs.groupby('Department')['Weekly hours'].aggregate(np.random.choice)
random

In [ ]:
new_jobs.loc[new_jobs['Job title'] == 'National Administration Analyst', 'Weekly hours'] = random['national administration']
new_jobs.loc[new_jobs['Job title'] == 'eCommerce Analyst','Weekly hours'] = random['ecommerce']
new_jobs.loc[new_jobs['Job title'] == 'eCommerce Associate Analyst','Weekly hours'] = random['ecommerce']

In [ ]:
print('Weekly hours: ', new_jobs['Weekly hours'].unique())

In [ ]:
new_jobs.loc[pd.isna(new_jobs['Years to upper salary']),]

In [ ]:
np.random.seed(111)
random1 = new_jobs.groupby('Department')['Years to upper salary'].aggregate(np.random.choice)
random1

In [ ]:
new_jobs.loc[new_jobs['Job title'] == 'National Administration Apprentice','Years to upper salary'] = random1['national administration']
new_jobs.loc[new_jobs['Job title'] == 'National IT Apprentice','Years to upper salary'] = random1['it']
new_jobs.loc[new_jobs['Job title'] == 'Quality Control Deputy','Years to upper salary'] = random1['quality control']

In [ ]:
print('Weekly hours: ', new_jobs['Weekly hours'].unique())

In [ ]:
new_jobs

# Featuring engineering 

Because we want to examine the possible effects of our predictors on the average salary we need to calculate a feature with the average from the upper and lower salary.

In [ ]:
average_salary = (new_jobs['Salary lower'] + new_jobs['Salary upper'])/2
average_salary

Also we will include in my regression the job title and examine it's effect of the average salary. To do this we need to transform the job titles from strings to numerical values. More specifically we want to examine how much is the effect of the job hierarchy titles in the average salary. So we will interpret the job titles as categorical values based on the hierarchy titles as follows for lower to higher:

- Apprentice
- Assistant (and also Technician / Deputy / Advisor)
- Analyst  (Technologist / Chef)
- Specialist
- Leader
- Scientist
- Manager
- Senior Manager

In [ ]:
title_num =[]
for title in new_jobs['Job title']:
    if 'Apprentice' in title:
        title_num.append(1)
    elif 'Assistant' in title:
        title_num.append(2)
    elif 'Technician' in title:
        title_num.append(2)  
    elif 'Deputy' in title:
        title_num.append(2)
    elif 'Advisor' in title:
        title_num.append(2)
    elif 'Analyst' in title:
        title_num.append(3)
    elif 'Chef' in title:
        title_num.append(3)
    elif 'Technologist' in title:
        title_num.append(3)
    elif 'Specialist' in title:
        title_num.append(4)
    elif 'Administrator' in title:
        title_num.append(4)
    elif 'Leader' in title:
        title_num.append(5)
    elif 'Co-ordinator' in title:
        title_num.append(5)
    elif 'Scientist' in title:
        title_num.append(6)
    elif 'Manager' in title:
        title_num.append(7)
    elif 'Senior Manager' in title:
        title_num.append(8) 

In [ ]:
data = pd.DataFrame({'average_salary':average_salary,'Years to upper salary': new_jobs['Years to upper salary'],
                     'Weekly hours': new_jobs['Weekly hours'],'Holiday weeks': new_jobs['Holiday weeks'],
                    'Job_title_Num': title_num})

In [ ]:
data

# Formulate a hypothesis about the data and test this hypothesis

Hypothesis: The annual average salary specified in one job posting can be a predicted based on the weekly working hours, the hierarchy level of the job, the number of years you need to progress into the upper salary and the holiday weeks specified in the job description. 

In [ ]:
import statsmodels.api as sm

In [ ]:
X =data[['Job_title_Num','Years to upper salary','Weekly hours','Holiday weeks']]
Y = data[['average_salary']]
X = sm.add_constant(X)
lm = sm.OLS(Y,X).fit()
print(lm.summary())

Statistical Interpretation of our results:

Based on the results of our regression we can see that our predictors (Job_title_Num, Years to upper salary, Weekly hours, Holiday weeks ) explain the 70.5% (based on the Adjusted R-squared) of the variability of the average salaries in the job postings. In Addition, we can reject the null hypothesis (that there is no linear relationship between my predictors and me response) and we can say that at least one predictor have a linear relationship with the average salaries. More specifically, we have to predictors that are statistically significant which are the weekly working hours and the numeric representation of Job titles. The Years to upper salary and Holiday weeks aren't statistically significant. 

Note: The extremely low p-value of the numeric representation of Job titles is probably to my interpretation of the order of the hierarchy status based on the salary so we can't assess it properly.  

Business Interpretation of our results:

- We can see that all predictors have positive coefficient with the average salary, so when the value of any predictor rise will also rise the average salary.

More specifically about the statistically significant predictors:

- If the weekly hours rise by 1 base point - meaning rise by 10 hours - this will result to £876.96 rise in the annual average salary.
- If the Job title rise by 1 base point - meaning rise by one hierarchy level (from Apprentice to Assistant)- this will result to £4843.57 rise in the annual average salary.

More specifically about the not statistically significant predictors:

- If the Years to upper salary rise by 1 base point - meaning rise by 1 year - this will result to £214.35 rise in the annual average salary.
- If the Holiday weeks rise by 1 base point - meaning rise by 1 week - this will result to £3661.10 rise in the annual average salary which looks kind of weird.

Because our first hypothesis contained some variables that didn't have a statistically significant result we will try a much simpler regression model with only the significant predictors (Weekly hours and Job title) and check if this has an impact of the R squared of the model.

New Hypothesis: The annual average salary affected by the weekly working hours and the hierarchy level of the job.

In [ ]:
X1 =data[['Job_title_Num','Weekly hours']]
Y = data[['average_salary']]
X1 = sm.add_constant(X1)
lm1 = sm.OLS(Y,X1).fit()
print(lm1.summary())

Statistical Interpretation of our results:

Based on the results of our second regression we can see that even with the reduced number of predictors (Job_title_Num, Weekly hours) the model still explains the 70% (based on the Adjusted R-squared) of the variability of the average salaries in the job postings. In Addition, we can still reject the null hypothesis and we can say that at least one of the two predictor have a linear relationship with the average salaries. Lastly, both our two predictors now have extremely small p-value so are still both statistically significant.

Business Interpretation of our results:

- Still we can see that our two predictors have positive coefficient with the average salary, so when the value of any predictor rise will also rise the average salary.

More specifically about the statistically significant predictors:

- If the weekly hours rise by 1 base point - meaning rise by 10 hours - this will result to £1032.82 rise in the annual average salary.
- If the Job title rise by 1 base point - meaning rise by one hierarchy level (from Apprentice to Assistant)- this will result to £4633.97 rise in the annual average salary.

Note: Due to the fact that we form our hypothesis that late in the process and not at the beginning the choice of the predictors and the results will contain some Bias. Probably we can avoid this Bias problem if we formulate our hypothesis in the beginning, even before web scrapping.

Checking about Correlation between our predictors:

In [ ]:
correlation_matrix = data.corr(method='pearson', min_periods=1)

In [ ]:
correlation_matrix

Result: Based on the correlation matrix we can see that 'Job_title_Num' predictor is highly correlated with the 'Weekly hours' and this can create a problem in both regression because we can't separate properly the effect of this to predictors in the response. So probably this is not the most appropriate regression model for us to use.